# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: **Triệu Nhật Minh**
- MSSV: **21127112**
- Lớp: **21CLC02**

# Import

In [285]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Latex
from sklearn.model_selection import KFold
# Import thêm dữ thư viện nếu cần

# Tạo dictionary chứa tên các cột

In [286]:
dict_columns = {1: 'Gender', 2: '10percentage', 3: '12percentage', 4: 'CollegeTier', 5: 'Degree', 6: 'collegeGPA', 7: 'CollegeCityTier', 8: 'English', 9: 'Logical', 10: 'Quant', 11: 'Domain', 12: 'ComputerProgramming', 13: 'ElectronicsAndSemicon', 14: 'ComputerScience', 15: 'MechanicalEngg', 16: 'ElectricalEngg', 17: 'TelecomEngg', 18: 'CivilEngg', 19: 'conscientiousness', 20: 'agreeableness', 21: 'extraversion', 22: 'nueroticism', 23: 'openess_to_experience', 24: 'Salary'}

# Đọc dữ liệu

In [287]:

# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần
# 1a
X_train_np = np.array(X_train)
y_train_np = np.array(y_train)

X_1a = X_train_np[:, :11]
y_1a = y_train_np

X_1a_test = np.array(test.iloc[:, :11])
y_1a_test = np.array(test.iloc[:, -1])

# 1b
X_1b = X_train.iloc[:, -5:]
X_1b_cons = np.array(X_1b.iloc[:, 0])
X_1b_agr = np.array(X_1b.iloc[:, 1])
X_1b_ext = np.array(X_1b.iloc[:, 2])
X_1b_nue = np.array(X_1b.iloc[:, 3])
X_1b_ope = np.array(X_1b.iloc[:, 4])

# 1c
X_1c = X_train.iloc[:, 7:10]
X_1c_eng = np.array(X_1c.iloc[:, 0])
X_1c_log = np.array(X_1c.iloc[:, 1])
X_1c_qua = np.array(X_1c.iloc[:, 2])

# Cài đặt hàm

In [288]:
# Cài đặt các hàm cần thiết ở đây

In [289]:
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T # 
        self.w = X_pinv @ y
        return self

    def get_params(self):
        return self.w
    
    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)   

In [290]:
def mae(y, y_hat):
    return np.mean(np.abs(y.ravel() - y_hat.ravel()))

In [291]:
def latex_text(params, dict=dict_columns):
    text = '$$\\text{Salary} = ' + str(params[0][0]) + ' \\times \\text{' + dict[1] + '}'
    for i in range(1, len(params)):
        text += str(params[i][0]) if params[i][0] < 0 else ' + ' + str(params[i][0])
        text += ' \\times \\text{' + dict[i+1] + '}'
        if i % 4 == 0:
            text += ' \\\\ '
    text += '$$'
    return text

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [292]:
# Phần code cho yêu cầu 1a

lr = OLSLinearRegression().fit(X_1a, y_1a)

params = lr.get_params().reshape(-1,1)

y_pred_1a = lr.predict(X_1a_test)

array([[-22756.51282098],
       [   804.50315567],
       [  1294.65456525],
       [-91781.89753061],
       [ 23182.38867918],
       [  1437.54867231],
       [ -8570.66198486],
       [   147.85829929],
       [   152.88847648],
       [   117.22184593],
       [ 34552.28622083]])

In [293]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
mae_1a = mae(y_1a_test, y_pred_1a)
mae_1a

104863.77754033108

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$

In [294]:
Latex(latex_text(params))

<IPython.core.display.Latex object>

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [295]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

# Shuffle k = 20 (k-fold cross validation)
k = 20
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Chuyển y_train thành numpy array
y_1b = np.array(y_train)

# Tạo mảng chứa các mảng MAE của các đặc trưng
mae_cons = []
mae_agr = []
mae_ext = []
mae_nue = []
mae_ope = []

### **conscientiousness**

In [296]:
for train_index, test_index in kf.split(X_1b_cons):
    X_folk_train, X_folk_test = X_1b_cons[train_index, None], X_1b_cons[test_index, None]
    y_folk_train, y_folk_test = y_1b[train_index, None], y_1b[test_index, None]
    y_folk_train = y_folk_train.reshape(-1,1)

    # Train the model
    lr = OLSLinearRegression()
    lr.fit(X_folk_train, y_folk_train)
    y_folk_pred = lr.predict(X_folk_test)
    
    # Calculate the error
    mae_cons.append(mae(y_folk_test, y_folk_pred))
    
print('MAE: ', np.mean(mae_cons))

MAE:  306221.755621708


### **agreeableness**

In [297]:
for train_index, test_index in kf.split(X_1b_agr):
    X_folk_train, X_folk_test = X_1b_agr[train_index, None], X_1b_agr[test_index, None]
    y_folk_train, y_folk_test = y_1b[train_index, None], y_1b[test_index, None]
    y_folk_train = y_folk_train.reshape(-1,1)

    # Train the model
    lr = OLSLinearRegression()
    lr.fit(X_folk_train, y_folk_train)
    y_folk_pred = lr.predict(X_folk_test)
    
    # Calculate the error
    mae_agr.append(mae(y_folk_test, y_folk_pred))
    
print('MAE: ', np.mean(mae_agr))

MAE:  300857.53034685535


### **extraversion**

In [298]:
for train_index, test_index in kf.split(X_1b_ext):
    X_folk_train, X_folk_test = X_1b_ext[train_index, None], X_1b_ext[test_index, None]
    y_folk_train, y_folk_test = y_1b[train_index, None], y_1b[test_index, None]
    y_folk_train = y_folk_train.reshape(-1,1)

    # Train the model
    lr = OLSLinearRegression()
    lr.fit(X_folk_train, y_folk_train)
    y_folk_pred = lr.predict(X_folk_test)
    
    # Calculate the error
    mae_ext.append(mae(y_folk_test, y_folk_pred))
    
print('MAE: ', np.mean(mae_ext))

MAE:  307059.3991665598


### **nueroticism**

In [299]:
for train_index, test_index in kf.split(X_1b_nue):
    X_folk_train, X_folk_test = X_1b_nue[train_index, None], X_1b_nue[test_index, None]
    y_folk_train, y_folk_test = y_1b[train_index, None], y_1b[test_index, None]
    y_folk_train = y_folk_train.reshape(-1,1)

    # Train the model
    lr = OLSLinearRegression()
    lr.fit(X_folk_train, y_folk_train)
    y_folk_pred = lr.predict(X_folk_test)
    
    # Calculate the error
    mae_nue.append(mae(y_folk_test, y_folk_pred))
    
print('MAE: ', np.mean(mae_nue))

MAE:  299376.1964789984


### **Openess_to_experience**

In [300]:
for train_index, test_index in kf.split(X_1b_ope):
    X_folk_train, X_folk_test = X_1b_ope[train_index, None], X_1b_ope[test_index, None]
    y_folk_train, y_folk_test = y_1b[train_index, None], y_1b[test_index, None]
    y_folk_train = y_folk_train.reshape(-1,1)

    # Train the model
    lr = OLSLinearRegression()
    lr.fit(X_folk_train, y_folk_train)
    y_folk_pred = lr.predict(X_folk_test)
    
    # Calculate the error
    mae_ope.append(mae(y_folk_test, y_folk_pred))
    
print('MAE: ', np.mean(mae_ope))

MAE:  303054.52630103764


In [301]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
# Đặc trưng tốt nhất: Nueroticism

X_1b_best_train = np.array(X_train['nueroticism'])
X_1b_best_train = X_1b_best_train[:, None]

best_personality_feature_model = OLSLinearRegression()
best_personality_feature_model.fit(X_1b_best_train, y_train)

# Đánh giá mô hình trên tập kiểm tra
X_1b_best_test = np.array(X_test['nueroticism'])
X_1b_best_test = X_1b_best_test[:, None]

params = best_personality_feature_model.get_params().reshape(-1, 1)
print('Params: ', params)

Params:  [[-56546.30375318]]


In [302]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
y_pred = best_personality_feature_model.predict(X_1b_best_test)

MAE = mae(y_test, y_pred)

print('MAE:', MAE)

MAE: 291019.693226953


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = -56546.304 \times \text{nueroticism}$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [303]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

# Shuffle k = 20 (k-fold cross validation)
k = 20
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Chuyển y_train thành numpy array
y_1c = np.array(y_train)

# Tạo mảng chứa các mảng MAE của các đặc trưng
mae_eng = []
mae_log = []
mae_qua = []

### **English**

In [304]:
for train_index, test_index in kf.split(X_1c_eng):
    X_folk_train, X_folk_test = X_1c_eng[train_index, None], X_1c_eng[test_index, None]
    y_folk_train, y_folk_test = y_1c[train_index, None], y_1c[test_index, None]
    y_folk_train = y_folk_train.reshape(-1,1)

    # Train the model
    lr = OLSLinearRegression()
    lr.fit(X_folk_train, y_folk_train)
    y_folk_pred = lr.predict(X_folk_test)
    
    # Calculate the error
    mae_eng.append(mae(y_folk_test, y_folk_pred))
    
print('MAE: ', np.mean(mae_eng))

MAE:  121901.64167007094


### **Logical**

In [305]:
for train_index, test_index in kf.split(X_1c_log):
    X_folk_train, X_folk_test = X_1c_log[train_index, None], X_1c_log[test_index, None]
    y_folk_train, y_folk_test = y_1c[train_index, None], y_1c[test_index, None]
    y_folk_train = y_folk_train.reshape(-1,1)

    # Train the model
    lr = OLSLinearRegression()
    lr.fit(X_folk_train, y_folk_train)
    y_folk_pred = lr.predict(X_folk_test)
    
    # Calculate the error
    mae_log.append(mae(y_folk_test, y_folk_pred))
    
print('MAE: ', np.mean(mae_log))

MAE:  120304.93112741373


### **Quant**

In [306]:
for train_index, test_index in kf.split(X_1c_qua):
    X_folk_train, X_folk_test = X_1c_qua[train_index, None], X_1c_qua[test_index, None]
    y_folk_train, y_folk_test = y_1c[train_index, None], y_1c[test_index, None]
    y_folk_train = y_folk_train.reshape(-1,1)

    # Train the model
    lr = OLSLinearRegression()
    lr.fit(X_folk_train, y_folk_train)
    y_folk_pred = lr.predict(X_folk_test)
    
    # Calculate the error
    mae_qua.append(mae(y_folk_test, y_folk_pred))
    
print('MAE: ', np.mean(mae_qua))

MAE:  118126.44763193058


In [307]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
# Đặc trưng tốt nhất: Quant

X_1c_best_train = np.array(X_train['Quant'])
X_1c_best_train = X_1c_best_train[:, None]

best_skill_feature_model = OLSLinearRegression()
best_skill_feature_model.fit(X_1c_best_train, y_train)

# Đánh giá mô hình trên tập kiểm tra
X_1c_best_test = np.array(X_test['Quant'])
X_1c_best_test = X_1c_best_test[:, None]

params = best_skill_feature_model.get_params().reshape(-1, 1)
print('Params: ', params)

Params:  [[585.89538084]]


In [308]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model
y_pred = best_skill_feature_model.predict(X_1c_best_test)

MAE = mae(y_test, y_pred)

print('MAE:', MAE)

MAE: 106819.57761989674


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 585.895 \times \text{Quant}$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [309]:
# Trình bày các phần tìm ra mô hình

## Thử nghiệm, so sánh các mô hình

In [310]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

In [311]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện

In [312]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$